<a href="https://colab.research.google.com/github/dgromann/cl_intro_ws2025/blob/main/tutorials/Tutorial3.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Tutorial 3: PyTorch, Tensors and Word Embeddings
This is the third tutorial with practical exercises for the lecture Introduction to Computational Linguistics in the winter semester 2025. Hands-on exercises are marked with 👋 ⚒ and questions are marked with ❓. Remember to first **store this notebook** in your Drive or GitHub.

---

## **Lesson 1: PyTorch Basics**

In order to get started with deep learning and learn how to write a code for neural networks from scratch, we need to familiarize ourselves with the packages that can be used to this end. There are two basic open source machine learning frameworks that can be used to this end:

*   TensorFlow (Google)
*   Torch (Facebook, Google DeepMind, Twitter)

Since these are high level core libraries, it is easier to use a framework that builds on top of it and adds some usability and documentation. We are going to work with [PyTorch](https://pytorch.org/). This first part of today's tutorial will introduce you to some core concepts of PyTorch before we start working with embeddings.

In [ ]:
# Let's first install pytorch
!pip install torch torchvision

The most basic and important concept in PyTorch is that of a **Tensor**. [What is a tensor?](https://en.wikipedia.org/wiki/Tensor). To speed up computation and offer more flexibility, PyTorch replaces numpy arrays with tensors.

In [ ]:
import torch
import numpy

# Seed for random number generator to ensure reproducibility of
# random initializations
torch.manual_seed(1)

#Difference between tensor and numpy array
a = torch.ones(5)
print("Tensor: ")
print(a)
print("Numpy array: ")
print(a.numpy(), "\n")


# This creates a randomly initialized 5 x 3 matrix
rand = torch.rand(5,3)
print("Randomly initialized tensor: ")
print(rand)
print("The shape of the matrix is its dimensions (here 5x3): ", rand.shape, "\n")

# This creates a 5 x 3 matrix filled with zeros and of dtype long
# There are eight datatypes in tensor, this one is a datatype of 64-bit integer (signed)
# Here are the others: https://pytorch.org/docs/stable/tensors.html
zeros = torch.zeros(4,3, dtype=torch.long)
print("Tensor initialized with zeros: ", zeros, "\n")

# Directly initialize a tensor with data
data = torch.tensor([[5.5, 3], [1, 2]])
print("Tensor initilialized with data: ", data, "\n")

# You can redefine an existing tensor
redefined = rand.new_ones(5, 3, dtype=torch.double)
print("Redefined randomly initialized tensor: ")
print(redefined)

redefined_too = torch.rand_like(redefined, dtype=torch.float)
print("\nInitializing randomly based on the size of redefined: ")
print(redefined_too)


# Get the size - this is actually a tuple that supports tuple operations
print(redefined_too.size())
#or
print(redefined.shape)

Tensors in torch also support basic operations:

In [ ]:
# Addition of tensors matching in size
x = torch.rand(5, 3)
y= torch.rand(5, 3)
print("Addition: ", x + y, "\n")
print("Addition alternative syntax: ", torch.add(x, y), "\n")

# Addition with providing a tensor as argument stores the result in "result"
result = torch.empty(5, 3)
torch.add(x, y, out=result)
print("Addition with tensor as argument: ", result, "\n")

Indexing operations of tensors follow the numpy standard:

In [ ]:
# Indexing
print("X: ", x, "\n")
print("Element at index one of each row of the matrix ", x[:, 1], "\n")

Resizing: if you wish to change the shape of the tensor you can use torch.view:

In [ ]:
# Exercise: resizing: what effect do the following
# resize operations have on the tensors
x = torch.randn(4, 4)
y = x.view(16)
z = x.view(-1, 8)
print("Resizing: ")
print("Original")
print(x)
print("Resized view(16)", y, "\n")
print("Resized view(-1, 8)", z, "\n")


One of the reasons why we use **tensors** is *vectorized operations*: operations that be conducted in parallel over a particular dimension of a tensor.

In [ ]:
data = torch.arange(1, 36, dtype=torch.float32).reshape(5, 7)
print("Data is:", data)

# We can perform operations like *sum* over each row...
print("Taking the sum over rows:")
print(data.sum(dim=1)) #(5,)

# or over each column.
print("Taking the sum over columns:")
print(data.sum(dim=0)) #(7,)

# Other operations are available:
print("Taking the standard deviation over rows:")
print(data.std(dim=1))

👋 ⚒ Write code that creates a `torch.tensor` with the following contents:
$\begin{bmatrix} 1 & 2.2 & 9.6 \\ 4 & -7.2 & 6.3 \end{bmatrix}$

Now compute the average of each row (`.mean()`) and each column.

What's the shape of the results?

In [ ]:
# Your code here

---

## **Lesson 2: Word Embeddings**

A vector representation of words trained with a neural network is called a word embedding. One of the most popular methods for training embeddings is called word2vec, which is a method for training embeddings from large natural language corpora.


`word2vec` literature:
  - Mikolov, T.,  Chen, K., Corrado, G., & Dean, J. (2013). [Efficient estimation of word representations in vector space](https://arxiv.org/abs/1301.3781). Corr abs/1301.3781.
  - Sutskever, I., Chen, K., Corrado, G. S., & Dean, J. (2013). [Distributed representations of words and phrases and their compositionally](https://papers.nips.cc/paper/5021-distributed-representations-of-words-and-phrases-and-their-compositionality.pdf). *Advances in neural information processing systems*. 2013.


- Other variants of embeddings training:
  - `fasttext` from Facebook
  - `GloVe` from Stanford NLP Group
- There are many ways to train word embeddings:
  - `gensim`: Simplest and straightforward implementation of `word2vec` and `GloVe`
  - Training based on deep learning packages (e.g., `keras`, `tensorflow`)
  - `spacy` (It comes with a pre-trained embeddings models, using GloVe.)

When multiplying a matrix on the hidden layer with the one-hot encoding, we obtain one row of the matrix. So the matrix serves as a lookup table for embeddings.

### Using Pre-Trained Embeddings

The code below exemplifies how to load a trained embedding model in the gensim library.

In [ ]:
# Let's first load a small subset of word2vec embeddings that have been trained on a
# large corpus of news documents
!wget https://github.com/dgromann/cl_intro_ws2025/raw/main/word2vec_embeddings.bin
!pip install gensim

In [ ]:
import gensim
import numpy as np

from sklearn.decomposition import PCA
from matplotlib import pyplot as plt

# Let's load the model
model = gensim.models.KeyedVectors.load_word2vec_format("word2vec_embeddings.bin", binary=True)

In [ ]:
# Print the length fo the whole vocabulary
print("Length of the vocabulary",len(model.key_to_index))

# Print the embedding of a specific word
print("Embedding for the word good: ", model["good"])


👋 ⚒ How many dimensions (numbers) does each vector in this trained embedding model have? Try to find this out with code, not by counting.

In [ ]:
#Your code goes here

Let's use embeddings to evaluate how similar two words are.

👋 ⚒ How can we get the first most similiar word of good from the list of the top 5 most similar words? In other words, how can you return and print only the word and not the tuple of word and cosine seimilarity?

In [ ]:
print(model.most_similar('good'))

# Get the top 5 most similar words of "good"
most_similar = model.most_similar("good", topn=5)
print(most_similar)

# Your code to get the first word from that list of top 5

We can also use these embeddings to obtain similar words in other pairs with the analogy task a is to b as c is to d, e.g. *man is to woman as king is to ?*

In [ ]:
# Check whether our embeddings are good at the analogy task
result = model.most_similar(positive=['woman', 'king'], negative=['man'], topn=1)
print(result)

👋 ⚒ Complete the following function `analogy` that takes three words in the analogy as input and should return the correct fourth result. For instance, good is to best as bad ist to?

So `analogy("good", "best", "bad")` should return the word ``"worst"``.

In [ ]:
def analogy(a, b, c):
  #Your code goes here


print(analogy("France", "Paris", "Austria"))
print(analogy("good", "best", "bad"))

We can use matplotlib to visualize the proximity of words in vector space.

In [ ]:
def display_pca_scatterplot(model, words):

    word_vectors = np.array([model[w] for w in words])

    twodim = PCA().fit_transform(word_vectors)[:,:2]

    plt.figure(figsize=(6,6))
    plt.scatter(twodim[:,0], twodim[:,1], edgecolors='k', c='r')
    for word, (x,y) in zip(words, twodim):
        plt.text(x+0.05, y+0.05, word)

In [ ]:
display_pca_scatterplot(model,
                        ['coffee', 'tea', 'beer', 'wine', 'water',
                         'hamburger', 'pizza',  'sushi', 'meatballs',
                         'dog', 'horse', 'cat', 'monkey', 'parrot', 'lizard',
                         'France', 'Germany', 'Hungary',
                         'school', 'college', 'university', 'institute'])

---

## **Lesson 3: Language Model and Embeddings in PyTorch**


In PyTorch, it is also possible to create your own neural network.

In [ ]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim

# Again random number generator to ensure reproducibility
torch.manual_seed(1)

We can use nn.Linear(H_in, H_out) to create a a linear layer. This will take a matrix of (N, *, H_in) dimensions and output a matrix of (N, *, H_out). The * denotes that there could be arbitrary number of dimensions in between. The linear layer performs the operation Ax+b, where A and b are initialized randomly. If we don't want the linear layer to learn the bias parameters, we can initialize our layer with bias=False.


In [ ]:
# Create the inputs
input = torch.ones(2,3,4)
print("Input ", input)

# N* H_in -> N*H_out
linear = nn.Linear(4, 2)
linear_output = linear(input)
linear_output


list(linear.parameters()) # Ax + b

Let's add an activation function:

In [ ]:
sigmoid = nn.Sigmoid()
output = sigmoid(linear_output)
output

Instead of creating intermediate layers and passing variables around, we can create a sequence:

In [ ]:
block = nn.Sequential(
    nn.Linear(4, 2),
    nn.Sigmoid()
)

input = torch.ones(2,3,4)
output = block(input)
output

## Training Word Embeddings with PyTorch

As a toy example, we will convert words to word embeddings. The preprocessing below should be done more elegantly and not as simply as below.

The below implementation is just a toy implementation. For a better version, see [this word2vec implementation in Pytorch](https://adoni.github.io/2017/11/08/word2vec-pytorch/)

In [ ]:
import string

# We will use Shakespeare Sonnet 2
test_sentence = """When forty winters shall besiege thy brow,
And dig deep trenches in thy beauty's field,
Thy youth's proud livery so gazed on now,
Will be a totter'd weed of small worth held:
Then being asked, where all thy beauty lies,
Where all the treasure of thy lusty days;
To say, within thine own deep sunken eyes,
Were an all-eating shame, and thriftless praise.
How much more praise deserv'd thy beauty's use,
If thou couldst answer 'This fair child of mine
Shall sum my count, and make my old excuse,'
Proving his beauty by succession thine!
This were to be new made when thou art old,
And see thy blood warm when thou feel'st it cold."""

training_sentence = test_sentence.translate(str.maketrans('', '', string.punctuation)).lower().split()
print(training_sentence)

Next let's find our vocabulary, i.e., all the unique words in the training data:

In [ ]:
vocabulary = set(w for w in training_sentence)
vocabulary

We introduce a special token, `<unk>`, to tackle the words that are out of vocabulary. We could pick another string for our unknown token if we wanted. The only requirement here is that our token should be unique: we should only be using this token for unknown words. We will also add this special token to our vocabulary.

In [ ]:
vocabulary.add("<unk>")

Now we will create the index for our vocabulary - one index to word and one word to index to make looking up words easier:

In [ ]:
ix_to_word = sorted(list(vocabulary))
word_to_ix = {word: ind for ind, word in enumerate(ix_to_word)}

👋 ⚒ How can we now lookup which word is the fifth word in our index list?

In [ ]:
# Your code here

👋 ⚒ How can we lookup the index for "within"?

In [ ]:
# Your code here

We will use a very simple solution of building trigrams to train our model.

In [ ]:
trigrams = [([training_sentence[i], training_sentence[i + 1]], training_sentence[i + 2])
            for i in range(len(training_sentence)-2)]
print(trigrams)

The following class defines the exact setup of our embedding model.

In [ ]:
class NGramLanguageModeler(nn.Module):

    def __init__(self, vocab_size, embedding_dim, window_size, hidden_dim):
        super(NGramLanguageModeler, self).__init__()

        self.embeddings = nn.Embedding(vocab_size, embedding_dim)
        self.hidden_layer = nn.Sequential(
            nn.Linear(window_size * embedding_dim, hidden_dim),
            nn.ReLU()
        )
        self.output_layer = nn.Linear(hidden_dim, vocab_size)

    def forward(self, inputs):
        embeds = self.embeddings(inputs).view((1, -1))
        layer1 = self.hidden_layer(embeds)
        output = self.output_layer(layer1)
        probabilities = nn.functional.log_softmax(output, dim=1)
        return probabilities

👋 ⚒ Now let's train the model. Try to adapt the hyperparamters so that the total_loss is reduced. These include in the initial settings:

*   Dimensionality of the embeddings: 10
*   Dimensionality of the hidden layer: 128
*   Number of epochs: 10
*   Learning rate: 0.002
*   Loss function (`NLLLoss()` Negative Log Likelihood right now)

The window size can in this toy example not be changed, since we train on trigrams (so we only have two context words in the set).

The code cell before, the `class NGramLanguageModeler(nn.Module)` defines the exact setup of the network. For the setup one thing to change could be the  activation functions (ReLU and Log-Softmax right now) - others can be found [here](https://pytorch.org/docs/stable/nn.html) for sequential layers and here for other uses as [nn.functional](https://pytorch.org/docs/stable/nn.functional.html)).

Just for comparison, this is the initial output with the first settings of the notebook:

`See how loss decreases with each epoch:  [4.502952638980561, 4.453238930322428, 4.404419487556525, 4.356399970771992, 4.309086953644204, 4.262444785210938, 4.216389432417608, 4.170906921403598, 4.125928629816106, 4.081349758975274]
Loss of the last epoch: 4.081349758975274`

In [ ]:
window_size = 2
embedding_dim = 10
hidden_dim = 128
num_epochs = 10

losses = []
# Negative log likelihood
loss_function = nn.NLLLoss()
ngram_model = NGramLanguageModeler(len(vocabulary), embedding_dim, window_size, hidden_dim)

# What do SGD and lr mean? What happenes if you change them?
optimizer = torch.optim.SGD(ngram_model.parameters(), lr=0.002)

for epoch in range(num_epochs):
    total_loss = 0
    for context, target in trigrams:
        # Step 1. Prepare the inputs to be passed to the model (i.e, turn the words
        # into integer indices and wrap them in tensors)
        context_idxs = torch.tensor([word_to_ix[w] for w in context], dtype=torch.long)

        # Step 2. Recall that torch *accumulates* gradients. Before passing in a
        # new instance, you need to zero out the gradients from the old
        # instance
        optimizer.zero_grad()

        # Step 3. Run the forward pass, getting probabilities over next
        # words - the size of this tensor is 87 corresponding to the size of the vocabulary
        probabilities = ngram_model.forward(context_idxs)

        # Step 4. Compute your loss function. The target word (gold standard label) needs to be
        # wrapped in a tensor.
        loss = loss_function(probabilities, torch.tensor([word_to_ix[target]], dtype=torch.long))

        # Step 5. Do the backward pass and update the gradient
        loss.backward()
        optimizer.step()

        # Get the Python number from a 1-element Tensor by calling tensor.item()
        total_loss += loss.item()
    losses.append(total_loss / len(trigrams))

print("See how loss decreases with each epoch: ", losses)
print("Loss of the last epoch:", losses[-1])

👋 ⚒ Implement the cosine simmilarity for the trained embeddings. For this you need to find the index of the two words you wish to compare, get their embeddings, convert them into numpy arrays, and run them through the cosine function provided.

In [ ]:
import numpy as np
from numpy.linalg import norm

# Tensor to look up specific embeddings
lookup_tensor = torch.tensor(list(word_to_ix.values()), dtype=torch.long)

# Embedding for the first word in the vocabulary
lookup_embeds = ngram_model.embeddings(lookup_tensor)[0]
print(lookup_embeds)

#cosine = np.dot(A,B)/(norm(A)*norm(B))